In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 29.1 MB/s 
     |████████████████████████████████| 6.6 MB 56.1 MB/s 
     |████████████████████████████████| 596 kB 66.0 MB/s 
     |████████████████████████████████| 101 kB 14.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#Import required Libraries 

import torch 
import torchtext 
import numpy as np 
import os 
import random 
import pandas as pd 
from tqdm.notebook import tqdm 
from sklearn.model_selection import train_test_split 
from transformers import BertTokenizer 
from torch.utils.data import TensorDataset 
from transformers import BertForSequenceClassification 
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler 
from sklearn.metrics import f1_score 

# access google drive
from google.colab import drive
drive.mount("/content/gdrive")
google_path =  '/content/gdrive/MyDrive/Social Media 2022/'

from os.path import exists # to check if file exists


Mounted at /content/gdrive


In [ ]:

filename = google_path + 'Sanitized datasets MFTC/df_Baltimore.csv'
if (exists(filename)): 
  df_all = pd.read_csv(filename)

print(df_all.head())

                                               tweet  annotation  \
0                                   nice try 's mine    betrayal   
1  rt keep saying conversation racism . wo n't pi...    betrayal   
2  coming 4 p.m. baltimore police will deliver up...   non-moral   
3  last night drove right protest sites baltimore...        care   
4                    thugs cops murdered freddiegray  subversion   

   Moral_nonMoral  
0               0  
1               0  
2               1  
3               0  
4               0  


In [ ]:
# check for NaNs

df_all.info()
print(df_all['tweet'].isnull().values.any())
print(df_all['tweet'].isnull().values)
print(df_all['tweet'].isnull().sum())

# remove NaNs
df_all = df_all.dropna()
df_all.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5593 entries, 0 to 5592
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet           5563 non-null   object
 1   annotation      5593 non-null   object
 2   Moral_nonMoral  5593 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 131.2+ KB
True
[False False False ... False False False]
30
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5563 entries, 0 to 5592
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet           5563 non-null   object
 1   annotation      5563 non-null   object
 2   Moral_nonMoral  5563 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 173.8+ KB


In [ ]:
# look how many Tweets in which classes we have
df_all.Moral_nonMoral.value_counts(dropna = False)

1    3265
0    2298
Name: Moral_nonMoral, dtype: int64

possible_labels = df_all.annotation.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

df_all['label'] = df_all.annotation.replace(label_dict)
df_all.head()

Split the dataset

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df_all.index.values, df_all.Moral_nonMoral.values, test_size=0.1, random_state=17, stratify=df_all.Moral_nonMoral.values) 

df_all['data_type'] = ['not_set']*df_all.shape[0]   #CREATING A NEW COLUMN IN DATASET AND SETTING ALL VALUES TO 'not_set' 

df_all.loc[X_train, 'data_type'] ='train' #CHECKING AND SETTING data_type TO TRAIN 
df_all.loc[X_val, 'data_type'] = 'val' #CHECKING AND SETTING data_type TO VAL


df_all.groupby(['Moral_nonMoral', 'data_type']).count() #TO CHECK WHICH CATEGORY DATA IS IN WHICH data_type

tweet  annotation
Moral_nonMoral data_type                   
0              train       2068        2068
               val          230         230
1              train       2938        2938
               val          327         327

use BERT-Base, Uncased Model which has 12 layers, 768 hidden, 12 heads, 110M parameters.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

#ENCODING DATA

encoded_data_train = tokenizer.batch_encode_plus(df_all[df_all.data_type=='train'].tweet.values,
                                                add_special_tokens=True,
                                                return_attention_mask=True,
                                               # pad_to_max_length=True,
                                               # max_length=256,
                                                truncation = True, 
                                                padding = "max_length",
                                                return_tensors='pt'
                                                )
encoded_data_val = tokenizer.batch_encode_plus(df_all[df_all.data_type=='val'].tweet.values,
                                               add_special_tokens=True,
                                               return_attention_mask=True,
                                               # pad_to_max_length=True,
                                               # max_length=256,
                                               truncation = True, 
                                               padding = "max_length",
                                               return_tensors='pt'
                                               )

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df_all[df_all.data_type=='train'].Moral_nonMoral.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df_all[df_all.data_type=='val'].Moral_nonMoral.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)

dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
#SETTING MODEL
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2, output_attentions=False, output_hidden_states=False ) #try only two category

#CREATING DATA LOADERS
dataloader_train = DataLoader(dataset_train,sampler = RandomSampler(dataset_train), batch_size=16) # 32)
                             

dataloader_val = DataLoader(dataset_val, sampler = RandomSampler(dataset_val), batch_size=16) #32)
      

#SETTING OPTIMIZERS

optimizer = AdamW(model.parameters(),lr=1e-5,eps=1e-8)

epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train)*epochs)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# Just right before the actual usage select your hardware
device = torch.device('cuda') # cuda or cpu
model = model.to(device)      # send your model to your hardware
torch.cuda.is_available(),torch.cuda.device_count()


(True, 1)

In [ ]:
from sklearn.metrics import accuracy_score
#FUNCTION TO CALCULATE F1 SCORE
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()
    return accuracy_score(labels_flat, preds_flat)



#FUNCTION FOR MODEL EVALUATION
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
# training model
# first empty cache
torch.cuda.empty_cache()

for epoch in tqdm(range(1, epochs+1)):
    model.train()
    
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc ='Epoch {:1d}'.format(epoch),
                        leave=False,
                        disable=False
                       )
    for batch in progress_bar:
        optimizer.zero_grad() # added for performance reasons (https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        
        inputs = { 'input_ids' : batch[0],
                 'attention_mask' : batch[1],
                 'labels' : batch[2]
                 }
        outputs = model(**inputs)
        
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss':'{:.3f}'.format(loss.item()/len(batch))})
    
    # THIS SECTION OF CODE IS JUST FOR PRINTING VALUES AFTER EACH EPOCH.
    torch.save(model.state_dict(), f'BERT_ft_epoch{epoch}.model')
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'Accuracy: {val_f1}')  

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/313 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.405222929538058
Validation loss: 0.39125349181038993
Accuracy: 0.8061041292639138


Epoch 2:   0%|          | 0/313 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.2798837863813384
Validation loss: 0.3452664294413158
Accuracy: 0.8491921005385996


...


Epoch 8
Training loss: 0.09079602934135951
Validation loss: 0.6545998671757324
Accuracy: 0.8689407540394973


Epoch 9:   0%|          | 0/313 [00:00<?, ?it/s]


Epoch 9
Training loss: 0.08024328850996726
Validation loss: 0.676615108137152
Accuracy: 0.8599640933572711


Epoch 10:   0%|          | 0/313 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.06730224767226738
Validation loss: 0.6892815340110766
Accuracy: 0.8617594254937163


In [ ]:
# look at accuracy

#_, predictions, true_val = evaluate(dataloader_val)  #why _ ? reason behind this is evaluate function return 3 values and i don't require the 1st value i.e., loss_val_avg

#accuracy_per_class(predictions, true_val)

In [ ]:
torch.save(model, '/content/gdrive/MyDrive/Social Media 2022/Model BERT/model_BERT_Baltimore.pt')

#FUNCTION FOR CALCULATING ACCURACY PER CLASS
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k,v in label_dict.items()}
    
    preds_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(label_dict_inverse[label])
        print("accuracy ", len(y_preds[y_preds==label])/len(y_true))